In [1]:
import os
import sys
import copy
import wget
import time
import asyncio
import warnings
import logging
from astropy import time as astropytime

import astropy.units as u
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

%matplotlib inline

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
attcs = ATTCS()

await attcs.start_task 

Read historical data in 2.03 sec
Read historical data in 3.04 sec
Read historical data in 3.37 sec
Read historical data in 3.97 sec
Read historical data in 4.99 sec
Read historical data in 6.00 sec
Read historical data in 7.16 sec
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 14 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 25 messages


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 15 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 18 messages


In [ ]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
await attcs.prepare_for_flatfield()

In [4]:
await attcs.shutdown()

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 19 messages
Disabling ATAOS corrections
RemoteEvent(ATMCS, 0, target) falling behind; read 35 messages
Disable ATDomeTrajectory
Slew telescope to Park position.
Flushing events
Sending command
Scheduling check coroutines
process as completed...
[Telescope] delta Alt = +019.755 | delta Az= -161.947
[Telescope] delta Alt = +019.664 | delta Az= -161.200
Failed to slew telescope to park position. Continuing...
'NoneType' object has no attribute 'private_sndStamp'
Traceback (most recent call last):
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 600, in shutdown
    wait_dome=False)
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 177, in point_azel
    raise e
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 174, in point_azel
    slew_timeout=slew_timeout)
  File "/ho

RuntimeError: Unable to put atmcs in STANDBY


In [ ]:
attcs.dome_slew_tolerance = Angle(5.*u.deg)

In [ ]:
await attcs.slew_icrs(ra="03 00 00", dec="-45 00 00", rot_pa=0., target_name="Test")

In [ ]:
await attcs.point_azel(target_name="Park position",
                       az=attcs.tel_park_az,
                       el=attcs.tel_park_el,
                       wait_dome=False)


In [ ]:
await attcs.startup(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
await attcs.ataos.cmd_enableCorrection.set_start(hexapod=True)

In [6]:
latiss = LATISS(attcs.domain)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 37 messages
Read historical data in 0.10 sec
Read historical data in 0.52 sec
Read historical data in 0.86 sec
Read historical data in 1.41 sec
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 10 messages


In [7]:
await latiss.start_task

[None, None, None, None]

In [ ]:
await salobj.set_summary_state(latiss.atheaderservice, salobj.State.ENABLED)

In [ ]:
await latiss.take_image(exptime=2., shutter=True, image_type="OBJECT", group_id=astropytime.Time.now().tai.isot)

In [ ]:
await latiss.take_image

In [ ]:
ack = await attcs.atdometrajectory.cmd_standby.start()

In [ ]:
print(ack.private_sndStamp)

In [ ]:
print(ack)

In [24]:
await salobj.set_summary_state(latiss.atarchiver, salobj.State.ENABLED, settingsToApply="normal")

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [11]:
bias = await latiss.take_bias(nbias=1)

Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [12]:
print(bias)

None


In [35]:
image_1 = await latiss.take_image(exptime=2., shutter=True, image_type="OBJECT", group_id=astropytime.Time.now().tai.isot)

Waiting for header service LFO before continuing


In [41]:
_, _, prefix, suffix = image_1.imageName.split("_")

In [ ]:
        prefix = tmp[2]  # dayobs without the dashes

        # Don't remember why I used int here... whitespace?
        # surely fixable but bigger fish.
        suffix = '{:05d}'.format(int(tmp[3].split('-')[0]))  # SEQNUM, but need to trim extra 0 in obsid
        intra_visitID = int((prefix + suffix))


In [42]:
prefix, suffix

('20200129', '000004')

In [43]:
# suffix = '{:05d}'.format(int(tmp[3].split('-')[0]))  # SEQNUM, but need to trim extra 0 in obsid
intra_visitID = int((prefix + suffix[1:]))

In [44]:
intra_visitID

2020012900004

RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 22 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 25 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 20 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 19 me

In [45]:
await latiss.take_bias(nbias=10)

Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
Waiting for header service LFO before continuing
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 12 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behin